In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
# !pip install --upgrade accelerate
# !pip show transformers accelerate bitsandbytes
# !pip install --upgrade transformers accelerate bitsandbytes
# !pip uninstall accelerate -y
# !pip install accelerate --no-cache-dir
# !pip install llama-index
!pip install llama-cpp-python --upgrade


     ---------------------------------------- 0.0/66.7 MB ? eta -:--:--
     ------ -------------------------------- 10.5/66.7 MB 65.5 MB/s eta 0:00:01
     ------------------- ------------------- 33.3/66.7 MB 92.0 MB/s eta 0:00:01
     -------------------------------- ------ 55.6/66.7 MB 98.4 MB/s eta 0:00:01
     --------------------------------------- 66.7/66.7 MB 88.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp312-cp312-win_amd64.whl size=3795411 sha256=e54ec73ae084327fd1a2c3d2a0717a2b83adf30b0e33dd8a773c175f


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from PIL import Image
import torch
import matplotlib.pyplot as plt
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from IPython.display import display_markdown
import llama_index

In [5]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Device name: NVIDIA GeForce RTX 4080 SUPER


In [11]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
device = torch.device("cuda", torch.cuda.current_device())
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

### **Normal Generation**

In [6]:
from transformers import pipeline

def generate_chat_template(model, tokenizer, sys_pmt="", prompt="", temperature=0.7, top_p=0.9, max_new_tokens=1000, do_sample=True):
    messages = [
        {"role": "system", "content": sys_pmt},
        {"role": "user", "content": prompt},
    ]
    
    pipeline_ = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
    
    # Ensure EOS and PAD tokens exist
    eos_token_id = tokenizer.eos_token_id or tokenizer.convert_tokens_to_ids("<|endoftext|>")
    pad_token_id = tokenizer.pad_token_id or eos_token_id  # Use EOS as a fallback
    
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    outputs = pipeline_(
        prompt, 
        max_new_tokens=max_new_tokens,
        eos_token_id=eos_token_id, 
        pad_token_id=pad_token_id,  # Explicitly set pad token
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p
    )
    
    return outputs[0]["generated_text"].split(prompt)[1]


In [7]:
# Paraphrasing Prompt Basic Example
system_prompt = """
You are a basic paraphrasing tool.
"""

prompt = """
Paraphrase the following text:

Central Analytics Support & Enablement (CASE) is our enterprise support platform for the ~20k data scientists, data analysts, MLOps engineers, BI developers, and other support teams. CASE includes multiple features such as "CASEY" the chatbot, an intent-driven (i.e. non-Gen AI/non-LLM) chatbot microservice built on MS Copilot SDKs. Additionally, CASE includes the Collective Intelligence (CI) framework, conditional ticketing system, and MS Graph-enabled search. All of this enables product and support teams to efficiently address customer needs, manage their product knowledge documentation, and handle product-related issues.
"""

output = generate_chat_template(
    model=model,
    tokenizer=tokenizer,
    sys_pmt=system_prompt,
    prompt=prompt,
    temperature=0.7,  # Allows for more variation
    top_p=0.9, # Nucleus sampling
    do_sample=True    # Enables sampling
)
display_markdown(output, raw=True)


Device set to use cuda:0


Here's a paraphrased version of the text:

Our enterprise support platform, Central Analytics Support & Enablement (CASE), is designed for ~20k data scientists, analysts, engineers, and support teams. CASE features a range of capabilities, including a unique chatbot, "CASEY," which utilizes intent-driven technology and is built on Microsoft Copilot SDKs. The platform also incorporates the Collective Intelligence framework, a conditional ticketing system, and MS Graph-enabled search functionality. These features allow product and support teams to effectively meet customer needs, manage product documentation, and address product-related issues efficiently.

### **ReAct Agent**

In [27]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-huggingface-api


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------  11.0/11.1 MB 68.9 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 49.5 MB/s eta 0:00:00
Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl (43.6 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
hf_token = "hf_"

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(
#     "meta-llama/Meta-Llama-3.1-8B-Instruct",
#     token=hf_token,
# )

# stopping_ids = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>"),
# ]

In [12]:
# import torch
# from llama_index.llms.huggingface import HuggingFaceLLM

# # Optional quantization to 4bit
# # import torch
# # from transformers import BitsAndBytesConfig

# # quantization_config = BitsAndBytesConfig(
# #     load_in_4bit=True,
# #     bnb_4bit_compute_dtype=torch.float16,
# #     bnb_4bit_quant_type="nf4",
# #     bnb_4bit_use_double_quant=True,
# # )

# llm = HuggingFaceLLM(
#     model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
#     model_kwargs={
#         "token": hf_token,
#         "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
#         # "quantization_config": quantization_config
#     },
#     generate_kwargs={
#         "do_sample": True,
#         "temperature": 0.6,
#         "top_p": 0.9,
#     },
#     tokenizer_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
#     tokenizer_kwargs={"token": hf_token},
#     stopping_ids=stopping_ids,
# )

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    token=hf_token,
)

# Load Model Manually
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
    device_map="auto",  # Auto-detect GPU if available
)

# Define Stopping IDs
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

# Pass Manually Loaded Model to HuggingFaceLLM
llm = HuggingFaceLLM(
    model=model,  # Use preloaded model
    tokenizer=tokenizer,  # Use preloaded tokenizer
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    stopping_ids=stopping_ids,
)

print("Model is successfully loaded on:", next(model.parameters()).device)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model is successfully loaded on: cpu


In [38]:
response = llm.complete("Who is Paul Graham?")

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Paul Graham is a British-American computer scientist, entrepreneur, and venture capitalist. He is the co-founder of Y Combinator, a well-known startup accelerator, and has been a prominent figure in the tech industry for over two decades.
Paul Graham was born on November 4, 1964, in London, England. He grew up in a family of modest means and developed an interest in computers at an early age. Graham earned a Bachelor of Arts degree in mathematics from the University of Illinois and later earned a Master of Science degree in computer science from the University of California, Berkeley.
Graham began his career as a programmer and later co-founded several companies, including Viaweb, a company that developed an online shopping cart for Amazon. In 1998, Viaweb was acquired by Yahoo! for $49 million. Graham used the funds from the acquisition to start Y Combinator, which has since become one of the most successful startup accelerators in the world.
Y Combinator has invested in over 2,000 s

In [39]:
!wget "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt" "paul_graham_essay.txt"

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [40]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["paul_graham_essay.txt"]
).load_data()

In [42]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [43]:
from llama_index.core import Settings

# bge embedding model
Settings.embed_model = embed_model

# Llama-3.1-8B-Instruct model
Settings.llm = llm

In [44]:
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(similarity_top_k=3)

In [45]:
response = query_engine.query("What did paul graham do growing up?")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


: 

In [ ]:
import json
from typing import Sequence, List

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)

In [ ]:
agent = ReActAgent.from_tools(
    [multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))